# Task 1 - Exploration of customer purchasing behavior

### Libraries to import:

In [5]:
import warnings
warnings.filterwarnings("ignore")
#Data Manipulation and Treatment
import numpy as np
import pandas as pd
from datetime import datetime
#Plotting and Visualizations
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from scipy import stats
import itertools
#dvc
import dvc.api
import mlflow

In [2]:
def str_to_date(date):
    return datetime.strptime(date, '%Y-%m-%d').date()

### Load the datasets 

In [10]:
def get_data(tag, Fpath, repo='G:/10_academy/Week_3/solution/Pharmaceutical-Sales-Prediction'):
    rev = tag
    path=Fpath
    data_url = dvc.api.get_url(path=path, repo=repo, rev=rev)
    df = pd.read_csv(data_url)
    return df

In [14]:
#The training Set
df_train=get_data('train_v1','data/train.csv')

FileNotFoundError: [Errno 2] No such file or directory: '..\\..\\..\\solutions\\temp\\d4\\7aedd9e2d580e06a6ef7ce1732e8b6'

In [12]:
#The training Set
df_test=get_data('test_v1','data/test.csv')

In [13]:
#The store Set
df_store=get_data('v1','data/store.csv')